##### Copyright 2020 The TensorFlow Authors. [Licensed under the Apache License, Version 2.0](#scrollTo=ByZjmtFgB_Y5).

In [ ]:
%install '.package(url: "https://github.com/tensorflow/swift-models", .branch("tensorflow-0.11"))' Datasets ImageClassificationModels
print("\u{001B}[2J")

In [ ]:
// #@title Licensed under the Apache License, Version 2.0 (the "License"); { display-mode: "form" }
// Licensed under the Apache License, Version 2.0 (the "License");
// you may not use this file except in compliance with the License.
// You may obtain a copy of the License at
//
// https://www.apache.org/licenses/LICENSE-2.0
//
// Unless required by applicable law or agreed to in writing, software
// distributed under the License is distributed on an "AS IS" BASIS,
// WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
// See the License for the specific language governing permissions and
// limitations under the License.

<table class="tfo-notebook-buttons" align="left">
 <td><a target="_blank" href="https://www.tensorflow.org/swift/tutorials/introducing_x10"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org で表示</a></td>
 <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/swift/blob/master/docs/site/tutorials/introducing_x10.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab で実行</a></td>
 <td><a target="_blank" href="https://github.com/tensorflow/swift/blob/master/docs/site/tutorials/introducing_x10.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub でソースを表示</a></td>
</table>

# X10 入門

Swift for TensorFlow は、デフォルトでは Eager execution を使用してテンソル演算を実行します。これは迅速なイテレーションを可能にしていますが、機械学習モデルのトレーニングで最もパフォーマンスが高いオプションというわけではありません。

[X10 テンソルライブラリ](https://github.com/tensorflow/swift-apis/blob/master/Documentation/X10/API_GUIDE.md)は、テンソルトレースと [XLA コンパイラ](https://www.tensorflow.org/xla)を利用して、Swift for TensorFlow に高性能なバックエンドを追加します。このチュートリアルでは、X10 を紹介し、トレーニングループを更新して GPU や TPU で実行するプロセスを案内します。

## Eager テンソルと X10 テンソル

Swift for TensorFlow の加速化計算は、テンソルの型を通して実行されます。テンソルは様々な演算に参加することができるため、機械学習モデルの基本的なビルディングブロックです。

デフォルトでは、Tensor は Eager execution を使用して演算ごとに計算を実行します。各 Tensor には、接続されているハードウェアとそれに使用されているバックエンドを説明する Device が関連づけられています。

In [ ]:
import TensorFlow
import Foundation

In [ ]:
let eagerTensor1 = Tensor([0.0, 1.0, 2.0])
let eagerTensor2 = Tensor([1.5, 2.5, 3.5])
let eagerTensorSum = eagerTensor1 + eagerTensor2
eagerTensorSum

In [ ]:
eagerTensor1.device

このノートブックを GPU 対応のインスタンスで実行している場合は、上記のデバイス説明内にハードウェアが反映されているはずです。Eager ランタイムは TPU をサポートしていないため、TPU をアクセラレータとして使用している場合には、ハードウェアターゲットとして CPU が使用されていることが分かります。

Tensor を作成する際に代替デバイスを指定して、Eager モードのデフォルトの デバイスを上書きすることができます。これが X10 バックエンドを使用した計算の実行をオプトインする方法です。

In [ ]:
let x10Tensor1 = Tensor([0.0, 1.0, 2.0], on: Device.defaultXLA)
let x10Tensor2 = Tensor([1.5, 2.5, 3.5], on: Device.defaultXLA)
let x10TensorSum = x10Tensor1 + x10Tensor2
x10TensorSum

In [ ]:
x10Tensor1.device

これを GPU 対応のインスタンスで実行している場合は、X10 テンソルのデバイス内にアクセラレータのリストが表示されます。これを Eager execution ではなく TPU 対応のインスタンスで実行している場合は、計算にそのデバイスを使用していることが確認できるはずです。X10は Swift for TensorFlow で TPU を活用する方法なのです。

デフォルトの Eager デバイスと X10 デバイスは、システム上の最初のアクセラレータを使用しようとします。GPU が接続されている場合、最初に利用可能な GPU を使用します。TPU が存在する場合は、X10 はデフォルトで最初の TPU コアを使用します。アクセラレータが見つからない、またはサポートされていない場合、デフォルトのデバイスは CPU にフォールバックします。

デフォルトの Eager や XLA デバイスを越えて、Device 内に固有のハードウェアターゲットとバックエンドターゲットを提供することができます。

In [ ]:
// let tpu1 = Device(kind: .TPU, ordinal: 1, backend: .XLA)
// let tpuTensor1 = Tensor([0.0, 1.0, 2.0], on: tpu1)

## Eager モードモデルをトレーニングする

デフォルトの Eager execution モードを使用し、モデルを設定してトレーニングする方法を見てみましょう。この例では [swift-models リポジトリ](https://github.com/tensorflow/swift-models)および MNIST 手書き数字分類データセットから単純な LeNet-5 モデルを使用します。

まず、MNIST データセットをセットアップしてダウンロードします。

In [ ]:
import Datasets

let epochCount = 5
let batchSize = 128
let dataset = MNIST(batchSize: batchSize)

次に、モデルとオプティマイザの設定を行います。

In [ ]:
import ImageClassificationModels

var eagerModel = LeNet()
var eagerOptimizer = SGD(for: eagerModel, learningRate: 0.1)

ここで、基本的な進捗状況の追跡とレポートを実装します。すべての中間統計はトレーニングが実行されるデバイスと同じデバイス上でテンソルとして保持され、`scalarized()` はレポート作成中にのみ呼び出されます。これで不必要に遅延テンソルがマテリアライズされるのを回避できるため、後で X10 を使用する場合に特に重要になります。

In [ ]:
struct Statistics {
    var correctGuessCount = Tensor<Int32>(0, on: Device.default)
    var totalGuessCount = Tensor<Int32>(0, on: Device.default)
    var totalLoss = Tensor<Float>(0, on: Device.default)
    var batches: Int = 0
    var accuracy: Float { 
        Float(correctGuessCount.scalarized()) / Float(totalGuessCount.scalarized()) * 100 
    } 
    var averageLoss: Float { totalLoss.scalarized() / Float(batches) }

    init(on device: Device = Device.default) {
        correctGuessCount = Tensor<Int32>(0, on: device)
        totalGuessCount = Tensor<Int32>(0, on: device)
        totalLoss = Tensor<Float>(0, on: device)
    }

    mutating func update(logits: Tensor<Float>, labels: Tensor<Int32>, loss: Tensor<Float>) {
        let correct = logits.argmax(squeezingAxis: 1) .== labels
        correctGuessCount += Tensor<Int32>(correct).sum()
        totalGuessCount += Int32(labels.shape[0])
        totalLoss += loss
        batches += 1
    }
}

最後に、トレーニングループを使用してモデルを 5 エポック分、実行します。

In [ ]:
print("Beginning training...")

for (epoch, batches) in dataset.training.prefix(epochCount).enumerated() {
    let start = Date()
    var trainStats = Statistics()
    var testStats = Statistics()
    
    Context.local.learningPhase = .training
    for batch in batches {
        let (images, labels) = (batch.data, batch.label)
        let 𝛁model = TensorFlow.gradient(at: eagerModel) { eagerModel -> Tensor<Float> in
            let ŷ = eagerModel(images)
            let loss = softmaxCrossEntropy(logits: ŷ, labels: labels)
            trainStats.update(logits: ŷ, labels: labels, loss: loss)
            return loss
        }
        eagerOptimizer.update(&eagerModel, along: 𝛁model)
    }

    Context.local.learningPhase = .inference
    for batch in dataset.validation {
        let (images, labels) = (batch.data, batch.label)
        let ŷ = eagerModel(images)
        let loss = softmaxCrossEntropy(logits: ŷ, labels: labels)
        testStats.update(logits: ŷ, labels: labels, loss: loss)
    }

    print(
        """
        [Epoch \(epoch)] \
        Training Loss: \(String(format: "%.3f", trainStats.averageLoss)), \
        Training Accuracy: \(trainStats.correctGuessCount)/\(trainStats.totalGuessCount) \
        (\(String(format: "%.1f", trainStats.accuracy))%), \
        Test Loss: \(String(format: "%.3f", testStats.averageLoss)), \
        Test Accuracy: \(testStats.correctGuessCount)/\(testStats.totalGuessCount) \
        (\(String(format: "%.1f", testStats.accuracy))%) \
        seconds per epoch: \(String(format: "%.1f", Date().timeIntervalSince(start)))
        """)
}

ご覧のように、モデルは期待通りにトレーニングされ、検証セットに対する精度はエポックごとに向上しました。Swift for TensorFlow モデルはこのようにして定義され、Eager execution を使用して実行されています。ここで、X10 を活用するためにはどのような修正が必要かを考えてみましょう。

## X10 モデルをトレーニングする

データセット、モデル、オプティマイザには、Eager execution デバイスのデフォルトで初期化されるテンソルが含まれています。X10 で作業をするには、これらのテンソルを X10 デバイスに移動する必要があります。

In [ ]:
let device = Device.defaultXLA
device

データセットについては、バッチがトレーニングループで処理されている時点で移動させ、Eager execution モデルからデータセットを再利用できるようにします。

モデルとオプティマイザについては、Eager execution デバイス上の内部テンソルで初期化して、X10 デバイスに移動させます。

In [ ]:
var x10Model = LeNet()
x10Model.move(to: device)

var x10Optimizer = SGD(for: x10Model, learningRate: 0.1)
x10Optimizer = SGD(copying: x10Optimizer, to: device)

トレーニングループに必要な修正は、いくつかの特定の時点で行います。まず最初に、トレーニングデータのバッチを X10 デバイスに移動させる必要があります。これは各バッチを取得する際に `Tensor(copy:to:)` を介して行います。

次の修正は、トレーニングループの実行中にトレースをどこで切断するかを示すことです。X10 はコードに必要なテンソル計算をトレースし、そのトレースの最適化された表現をジャストインタイムでコンパイルすることによって動作します。トレーニングループの場合は同じ演算を何度も繰り返すことになるので、トレース、コンパイル、再利用には理想的なセクションです。

Tensor からの値を明示的に要求するコード（これらは通常 `.scalars` または `.scalarized()` なので目立ちます）がない場合、X10 はすべてのループイテレーションをまとめてコンパイルしようとします。これを防ぎ、特定の時点でトレースを切断するために、オプティマイザがモデル重みを更新して検証中に損失と精度を取得した後で、明示的な `LazyTensorBarrier()` を配置します。これにより、トレーニングループの各ステップと検証中の推論の各バッチから成る 2 つの再利用トレースが作成されます。

これらの修正によって、以下のトレーニングループが発生します。

In [ ]:
print("Beginning training...")

for (epoch, batches) in dataset.training.prefix(epochCount).enumerated() {
    let start = Date()
    var trainStats = Statistics(on: device)
    var testStats = Statistics(on: device)
    
    Context.local.learningPhase = .training
    for batch in batches {
        let (eagerImages, eagerLabels) = (batch.data, batch.label)
        let images = Tensor(copying: eagerImages, to: device)
        let labels = Tensor(copying: eagerLabels, to: device)
        let 𝛁model = TensorFlow.gradient(at: x10Model) { x10Model -> Tensor<Float> in
            let ŷ = x10Model(images)
            let loss = softmaxCrossEntropy(logits: ŷ, labels: labels)
            trainStats.update(logits: ŷ, labels: labels, loss: loss)
            return loss
        }
        x10Optimizer.update(&x10Model, along: 𝛁model)
        LazyTensorBarrier()
    }

    Context.local.learningPhase = .inference
    for batch in dataset.validation {
        let (eagerImages, eagerLabels) = (batch.data, batch.label)
        let images = Tensor(copying: eagerImages, to: device)
        let labels = Tensor(copying: eagerLabels, to: device)
        let ŷ = x10Model(images)
        let loss = softmaxCrossEntropy(logits: ŷ, labels: labels)
        LazyTensorBarrier()
        testStats.update(logits: ŷ, labels: labels, loss: loss)
    }

    print(
        """
        [Epoch \(epoch)] \
        Training Loss: \(String(format: "%.3f", trainStats.averageLoss)), \
        Training Accuracy: \(trainStats.correctGuessCount)/\(trainStats.totalGuessCount) \
        (\(String(format: "%.1f", trainStats.accuracy))%), \
        Test Loss: \(String(format: "%.3f", testStats.averageLoss)), \
        Test Accuracy: \(testStats.correctGuessCount)/\(testStats.totalGuessCount) \
        (\(String(format: "%.1f", testStats.accuracy))%) \
        seconds per epoch: \(String(format: "%.1f", Date().timeIntervalSince(start)))
        """)
}

X10 バックエンドを使用したモデルトレーニングは、以前の Eager execution モデルの場合と全く同じように進行するはずです。最初のバッチの前と最初のエポックの終わりに遅延が生じる可能性がありますが、これはその時点で固有のトレースをジャストインタイムでコンパイルしているためです。アクセラレータを接続して実行している場合は、その時点以降のトレーニングが Eager モードよりも速く進んでいることが分かるはずです。

初期トレースのコンパイル時間と高速なスループットのトレードオフがありますが、大部分の機械学習モデルでは、繰り返し演算によるスループットの増加がコンパイルのオーバーヘッドを相殺するよりも大きくなるはずです。実際に複数のトレーニングケースにおいて、X10 を使用することによってスループットが 4 倍以上向上しています。

先に述べたとおり、X10 を使用すると TPU での作業ができるようになるだけでなく、作業が容易で、Swift for TensorFlow モデルのアクセラレータのすべてのクラスを使用できるようになります。